In [156]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import polars as pl
import glob
import joblib

In [157]:
# download data from kaggle
!kaggle competitions download -c ieee-fraud-detection
# unzip downloaded data
!unzip ieee-fraud-detection.zip -d ieee-fraud-detection



  0%|          | 0.00/118M [00:00<?, ?B/s]
  1%|          | 1.00M/118M [00:01<02:51, 715kB/s]
  2%|▏         | 2.00M/118M [00:02<02:05, 969kB/s]
  3%|▎         | 3.00M/118M [00:02<01:34, 1.28MB/s]
  3%|▎         | 4.00M/118M [00:03<01:16, 1.56MB/s]
  4%|▍         | 5.00M/118M [00:03<01:05, 1.81MB/s]
  5%|▌         | 6.00M/118M [00:03<00:55, 2.10MB/s]
  6%|▌         | 7.00M/118M [00:04<00:49, 2.35MB/s]
  7%|▋         | 8.00M/118M [00:04<00:45, 2.55MB/s]
  8%|▊         | 9.00M/118M [00:04<00:41, 2.76MB/s]
  8%|▊         | 10.0M/118M [00:05<00:38, 2.93MB/s]
  9%|▉         | 11.0M/118M [00:05<00:36, 3.08MB/s]
 10%|█         | 12.0M/118M [00:05<00:35, 3.17MB/s]
 11%|█         | 13.0M/118M [00:06<00:33, 3.31MB/s]
 12%|█▏        | 14.0M/118M [00:06<00:31, 3.44MB/s]
 13%|█▎        | 15.0M/118M [00:06<00:30, 3.53MB/s]
 14%|█▎        | 16.0M/118M [00:06<00:29, 3.65MB/s]
 14%|█▍        | 17.0M/118M [00:07<00:28, 3.78MB/s]
 15%|█▌        | 18.0M/118M [00:07<00:26, 3.94MB/s]
 16%|█▌        | 19.0M

In [134]:
file_list = glob.glob("ieee-fraud-detection/*_transaction.csv") + glob.glob("ieee-fraud-detection/*_identity.csv") 
file_list

['ieee-fraud-detection\\test_transaction.csv',
 'ieee-fraud-detection\\train_transaction.csv',
 'ieee-fraud-detection\\test_identity.csv',
 'ieee-fraud-detection\\train_identity.csv']

In [135]:
test_transaction, train_transaction, test_identity, train_identity = [pl.read_csv(file) for file in file_list]
test_identity = test_identity.rename({col: col.replace('-', '_') for col in test_identity.columns})

In [16]:
train_transaction.head()

TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,…,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
i64,i64,i64,f64,str,i64,f64,f64,str,f64,str,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2987000,0,86400,68.5,"""W""",13926,null,150.0,"""discover""",142.0,"""credit""",315.0,87.0,19.0,null,null,null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,null,13.0,null,null,null,…,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987001,0,86401,29.0,"""W""",2755,404.0,150.0,"""mastercard""",102.0,"""credit""",325.0,87.0,null,null,"""gmail.com""",null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,null,null,0.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987002,0,86469,59.0,"""W""",4663,490.0,150.0,"""visa""",166.0,"""debit""",330.0,87.0,287.0,null,"""outlook.com""",null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,null,null,0.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987003,0,86499,50.0,"""W""",18132,567.0,150.0,"""mastercard""",117.0,"""debit""",476.0,87.0,null,null,"""yahoo.com""",null,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,null,…,0.0,0.0,1.0,50.0,1758.0,925.0,0.0,354.0,0.0,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987004,0,86506,50.0,"""H""",4497,514.0,150.0,"""mastercard""",102.0,"""credit""",420.0,87.0,null,null,"""gmail.com""",null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,null,null,null,null,null,…,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
train_transaction.describe()

statistic,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,…,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
str,f64,f64,f64,f64,str,f64,f64,f64,str,f64,str,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",590540.0,590540.0,590540.0,590540.0,"""590540""",590540.0,581607.0,588975.0,"""588963""",586281.0,"""588969""",524834.0,524834.0,238269.0,37627.0,"""496084""","""137291""",590540.0,590540.0,590540.0,590540.0,590540.0,590540.0,590540.0,590540.0,590540.0,590540.0,590540.0,590540.0,590540.0,590540.0,589271.0,309743.0,327662.0,421618.0,280699.0,…,590528.0,590528.0,590528.0,590528.0,590528.0,590528.0,590528.0,590528.0,590528.0,590528.0,589271.0,589271.0,589271.0,590528.0,590528.0,590528.0,590528.0,590528.0,590528.0,82351.0,82351.0,82351.0,82351.0,82351.0,82351.0,82351.0,82351.0,82351.0,82351.0,82351.0,82351.0,82351.0,82351.0,82351.0,82351.0,82351.0,82351.0
"""null_count""",0.0,0.0,0.0,0.0,"""0""",0.0,8933.0,1565.0,"""1577""",4259.0,"""1571""",65706.0,65706.0,352271.0,552913.0,"""94456""","""453249""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1269.0,280797.0,262878.0,168922.0,309841.0,…,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,1269.0,1269.0,1269.0,12.0,12.0,12.0,12.0,12.0,12.0,508189.0,508189.0,508189.0,508189.0,508189.0,508189.0,508189.0,508189.0,508189.0,508189.0,508189.0,508189.0,508189.0,508189.0,508189.0,508189.0,508189.0,508189.0
"""mean""",3282269.5,0.03499,7.3723e6,135.027176,null,9898.734658,362.555488,153.194925,null,199.278897,null,290.733794,86.80063,118.50218,231.855423,null,null,14.092458,15.269734,0.005644,4.092185,5.571526,9.071082,2.848478,5.144574,4.48024,5.240343,10.241521,4.076227,32.539918,8.295215,94.347568,169.563231,28.343348,140.002441,42.335965,…,0.28314,0.264208,1.000007,139.748713,408.682375,230.41318,10.995986,118.195658,4.202175,39.17391,21.351473,43.319174,26.806977,109.818544,247.606741,162.153398,18.372476,42.073133,28.326584,6.220289,13.103775,9.184612,0.058494,0.85104,0.296633,0.33679,1.312844,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
"""std""",170474.358321,0.183755,4.6172e6,239.162522,null,4901.170153,157.793246,11.336444,null,41.244453,null,101.741072,2.690623,371.872026,529.053494,null,null,133.569018,154.668899,0.150536,68.848459,25.786976,71.508467,61.727304,95.378574,16.674897,95.581443,94.336292,86.666218,129.364844,49.544262,157.660387,177.315865,62.384721,191.096774,89.000144,…,0.623608,0.528238,0.002603,2348.849634,4391.992977,3021.924247,116.254277,352.983093,102.374938,172.128339,95.90297,173.619028,116.853222,2270.033202,3980.042828,2793.343636,332.304848,473.499307,382.053171,56.022561,106.739813,73.627893,0.304415,3.950295,1.364356,1.580144,8.769083,4.727971,6217.223583,11169.275702,7955.735482,243.861391,387.62948,274.57692,668.486833,1095.034387,814.946722
"""min""",2.987e6,0.0,86400.0,0.251,"""C""",1000.0,100.0,100.0,"""american express""",100.0,"""charge card""",100.0,10.0,0.0,0.0,"""aim.com""","""aim.com""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-122.0,0.0,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",3.134635e6,0.0,3.027065e6,43.321,null,6019.0,214.0,150.0,null,166.0,null,204.0,87.0,3.0,7.0,null,null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,26.0,1.0,0.0,1.0,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [18]:
train_identity.describe()

statistic,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,str,str,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,str,str,str,str,str,f64,str,str,str,str,str,str,str,str
"""count""",144233.0,144233.0,140872.0,66324.0,66324.0,136865.0,136865.0,5155.0,5155.0,74926.0,74926.0,140978.0,"""144233""",127320.0,80044.0,"""140985""","""129340""",139369.0,45113.0,139318.0,139261.0,5159.0,5169.0,"""5169""",4747.0,5132.0,5163.0,"""5169""","""140978""","""140978""","""77565""","""140282""",77586.0,"""73289""","""77805""","""140985""","""140985""","""140985""","""140985""","""140810""","""118666"""
"""null_count""",0.0,0.0,3361.0,77909.0,77909.0,7368.0,7368.0,139078.0,139078.0,69307.0,69307.0,3255.0,"""0""",16913.0,64189.0,"""3248""","""14893""",4864.0,99120.0,4915.0,4972.0,139074.0,139064.0,"""139064""",139486.0,139101.0,139070.0,"""139064""","""3255""","""3255""","""66668""","""3951""",66647.0,"""70944""","""66428""","""3248""","""3248""","""3248""","""3248""","""3423""","""25567"""
"""mean""",3.2363e6,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,99.745325,null,48.053071,-344.507146,null,null,189.451377,14.237337,353.128174,403.882666,368.26982,16.002708,null,12.800927,329.608924,149.070308,null,null,null,null,null,26.508597,null,null,null,null,null,null,null,null
"""std""",178849.571186,14.347949,159651.816856,0.598231,0.701015,5.249856,16.491104,11.384207,26.084899,0.983842,2.789446,1.127602,null,11.774858,93.695502,null,null,30.37536,1.561302,141.095343,152.160327,198.847038,6.897665,null,2.372447,97.461089,32.101995,null,null,null,null,null,3.737502,null,null,null,null,null,null,null,null
"""min""",2.987004e6,-100.0,1.0,-13.0,-28.0,-72.0,-100.0,-46.0,-100.0,-36.0,-100.0,90.0,"""Found""",10.0,-660.0,"""Found""","""Found""",100.0,10.0,100.0,100.0,100.0,10.0,"""IP_PROXY:ANONYMOUS""",11.0,100.0,100.0,"""Found""","""Found""","""Found""","""Android""","""BLU/Dash""",0.0,"""0x0""","""match_status:-1""","""F""","""F""","""F""","""F""","""desktop""","""0PAJ5"""
"""25%""",3.077142e6,-10.0,67992.0,0.0,0.0,0.0,-6.0,5.0,-48.0,0.0,0.0,100.0,null,49.0,-360.0,null,null,166.0,13.0,266.0,256.0,252.0,14.0,null,11.0,321.0,119.0,null,null,null,null,null,24.0,null,null,null,null,null,null,null,null
"""50%""",3.198818e6,-5.0,125803.0,0.0,0.0,0.0,0.0,14.0,-34.0,0.0,0.0,100.0,null,52.0,-300.0,null,null,166.0,15.0,341.0,472.0,252.0,14.0,null,11.0,321.0,149.0,null,null,null,null,null,24.0,null,null,null,null,null,null,null,null
"""75%""",3.392923e6,-5.0,228748.0,0.0,0.0,1.0,0.0,22.0,-23.0,0.0,0.0,100.0,null,52.0,-300.0,null,null,225.0,15.0,427.0,533.0,487.0,14.0,null,15.0,371.0,169.0,null,null,null,null,null,32.0,null,null,null,null,null,null,null,null
"""max""",3.577534e6,0.0,999595.0,10.0,0.0,52.0,0.0,61.0,0.0,25.0,0.0,100.0,"""NotFound""",64.0,720.0,"""Unknown""","""NotFound""",229.0,29.0,671.0,661.0,854.0,44.0,"""IP_PROXY:TRANSPARENT""",26.0,548.0,216.0,"""NotFound""","""New""","""NotFound""","""other""","""waterfox""",32.0,"""976x600""","""match_status:2""","""T""","""T""","""T""","""T""","""mobile""","""xs-Z47b7VqTMxs"""


In [19]:
train_identity.describe()

statistic,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,str,str,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,str,str,str,str,str,f64,str,str,str,str,str,str,str,str
"""count""",144233.0,144233.0,140872.0,66324.0,66324.0,136865.0,136865.0,5155.0,5155.0,74926.0,74926.0,140978.0,"""144233""",127320.0,80044.0,"""140985""","""129340""",139369.0,45113.0,139318.0,139261.0,5159.0,5169.0,"""5169""",4747.0,5132.0,5163.0,"""5169""","""140978""","""140978""","""77565""","""140282""",77586.0,"""73289""","""77805""","""140985""","""140985""","""140985""","""140985""","""140810""","""118666"""
"""null_count""",0.0,0.0,3361.0,77909.0,77909.0,7368.0,7368.0,139078.0,139078.0,69307.0,69307.0,3255.0,"""0""",16913.0,64189.0,"""3248""","""14893""",4864.0,99120.0,4915.0,4972.0,139074.0,139064.0,"""139064""",139486.0,139101.0,139070.0,"""139064""","""3255""","""3255""","""66668""","""3951""",66647.0,"""70944""","""66428""","""3248""","""3248""","""3248""","""3248""","""3423""","""25567"""
"""mean""",3.2363e6,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.69871,13.285354,-38.600388,0.091023,-0.301124,99.745325,null,48.053071,-344.507146,null,null,189.451377,14.237337,353.128174,403.882666,368.26982,16.002708,null,12.800927,329.608924,149.070308,null,null,null,null,null,26.508597,null,null,null,null,null,null,null,null
"""std""",178849.571186,14.347949,159651.816856,0.598231,0.701015,5.249856,16.491104,11.384207,26.084899,0.983842,2.789446,1.127602,null,11.774858,93.695502,null,null,30.37536,1.561302,141.095343,152.160327,198.847038,6.897665,null,2.372447,97.461089,32.101995,null,null,null,null,null,3.737502,null,null,null,null,null,null,null,null
"""min""",2.987004e6,-100.0,1.0,-13.0,-28.0,-72.0,-100.0,-46.0,-100.0,-36.0,-100.0,90.0,"""Found""",10.0,-660.0,"""Found""","""Found""",100.0,10.0,100.0,100.0,100.0,10.0,"""IP_PROXY:ANONYMOUS""",11.0,100.0,100.0,"""Found""","""Found""","""Found""","""Android""","""BLU/Dash""",0.0,"""0x0""","""match_status:-1""","""F""","""F""","""F""","""F""","""desktop""","""0PAJ5"""
"""25%""",3.077142e6,-10.0,67992.0,0.0,0.0,0.0,-6.0,5.0,-48.0,0.0,0.0,100.0,null,49.0,-360.0,null,null,166.0,13.0,266.0,256.0,252.0,14.0,null,11.0,321.0,119.0,null,null,null,null,null,24.0,null,null,null,null,null,null,null,null
"""50%""",3.198818e6,-5.0,125803.0,0.0,0.0,0.0,0.0,14.0,-34.0,0.0,0.0,100.0,null,52.0,-300.0,null,null,166.0,15.0,341.0,472.0,252.0,14.0,null,11.0,321.0,149.0,null,null,null,null,null,24.0,null,null,null,null,null,null,null,null
"""75%""",3.392923e6,-5.0,228748.0,0.0,0.0,1.0,0.0,22.0,-23.0,0.0,0.0,100.0,null,52.0,-300.0,null,null,225.0,15.0,427.0,533.0,487.0,14.0,null,15.0,371.0,169.0,null,null,null,null,null,32.0,null,null,null,null,null,null,null,null
"""max""",3.577534e6,0.0,999595.0,10.0,0.0,52.0,0.0,61.0,0.0,25.0,0.0,100.0,"""NotFound""",64.0,720.0,"""Unknown""","""NotFound""",229.0,29.0,671.0,661.0,854.0,44.0,"""IP_PROXY:TRANSPARENT""",26.0,548.0,216.0,"""NotFound""","""New""","""NotFound""","""other""","""waterfox""",32.0,"""976x600""","""match_status:2""","""T""","""T""","""T""","""T""","""mobile""","""xs-Z47b7VqTMxs"""


In [20]:
test_transaction.describe()

statistic,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,…,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
str,f64,f64,f64,str,f64,f64,f64,str,f64,str,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",506691.0,506691.0,506691.0,"""506691""",506691.0,498037.0,503689.0,"""503605""",502144.0,"""503684""",441082.0,441082.0,215474.0,36436.0,"""437499""","""135870""",506688.0,506688.0,506688.0,506688.0,506688.0,506688.0,506688.0,506688.0,506688.0,506688.0,506688.0,506688.0,501943.0,506688.0,500660.0,271922.0,303549.0,429840.0,282316.0,124783.0,…,506688.0,506688.0,506688.0,506688.0,506688.0,506688.0,506688.0,506688.0,506688.0,506688.0,500660.0,500660.0,500660.0,506688.0,506688.0,506688.0,506688.0,506688.0,506688.0,76431.0,76431.0,76431.0,76431.0,76431.0,76431.0,76431.0,76431.0,76431.0,76431.0,76431.0,76431.0,76431.0,76431.0,76431.0,76431.0,76431.0,76431.0
"""null_count""",0.0,0.0,0.0,"""0""",0.0,8654.0,3002.0,"""3086""",4547.0,"""3007""",65609.0,65609.0,291217.0,470255.0,"""69192""","""370821""",3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,4748.0,3.0,6031.0,234769.0,203142.0,76851.0,224375.0,381908.0,…,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,6031.0,6031.0,6031.0,3.0,3.0,3.0,3.0,3.0,3.0,430260.0,430260.0,430260.0,430260.0,430260.0,430260.0,430260.0,430260.0,430260.0,430260.0,430260.0,430260.0,430260.0,430260.0,430260.0,430260.0,430260.0,430260.0
"""mean""",3.916894e6,2.6930e7,134.725568,null,9957.222175,363.735379,153.543409,null,200.162975,null,291.846514,86.723412,87.06527,237.175047,null,null,10.093211,10.714635,0.027403,2.385875,4.962701,6.854992,1.678173,1.893994,4.611943,1.810149,7.484594,2.649486,27.816035,6.08389,108.207504,188.666621,33.394727,175.060799,50.977752,82.443145,…,0.321849,0.301547,1.000002,83.783107,259.371829,136.949416,12.838856,127.198552,5.153783,42.733066,21.238136,43.269524,26.730428,42.077887,83.808337,56.090776,27.620156,46.835629,36.772511,0.436681,1.216182,0.693514,0.047926,0.482658,0.163271,0.220486,0.460572,0.3099,315.128613,440.464333,381.102129,24.70803,58.339468,35.485643,99.05876,155.578153,139.814706
"""std""",146269.23696,4.7565e6,245.779822,null,4884.960969,158.688653,12.443013,null,40.562461,null,102.06273,2.987328,314.131694,556.450834,null,null,79.334782,88.896921,0.227753,41.942977,25.462195,46.783565,37.874827,26.766115,21.28264,23.692393,61.322505,53.97306,101.114033,28.315616,176.964526,197.099611,82.55821,250.41747,116.829786,192.867671,…,0.639583,0.555264,0.001405,2706.035763,3643.584651,2976.740478,337.427648,606.911563,278.305451,467.654628,94.662615,195.591901,115.676311,2249.338097,2919.418027,2263.811948,999.900445,1514.927745,1486.814768,2.129226,5.925144,3.343788,0.282497,2.555532,0.850853,1.789616,4.228813,2.632181,9494.045099,10648.4296,10395.559802,842.913287,990.361078,904.093921,2436.074522,4544.303974,4493.728267
"""min""",3.663549e6,1.8403224e7,0.018,"""C""",1001.0,100.0,100.0,"""american express""",100.0,"""charge card""",100.0,10.0,0.0,0.0,"""aim.com""","""aim.com""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",3.790222e6,2.2771551e7,40.0,null,6019.0,207.0,150.0,null,166.0,null,204.0,87.0,3.0,7.0,null,null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,26.0,1.0,0.0,0.0,0.0,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [21]:
print("training data")
print(train_transaction["isFraud"].value_counts(sort=True))
print("number of training data (transaction):", len(train_transaction))
print(train_transaction["TransactionID"].value_counts(sort=True))
print("number of training data (identity):", len(train_identity))
print(train_identity["TransactionID"].value_counts(sort=True))
print("number of test data (transaction):", len(test_transaction))
print(test_transaction["TransactionID"].value_counts(sort=True))
print("number of test data (identity):", len(test_identity))
print(test_identity["TransactionID"].value_counts(sort=True))


training data
shape: (2, 2)
┌─────────┬────────┐
│ isFraud ┆ count  │
│ ---     ┆ ---    │
│ i64     ┆ u32    │
╞═════════╪════════╡
│ 0       ┆ 569877 │
│ 1       ┆ 20663  │
└─────────┴────────┘
number of training data (transaction): 590540
shape: (590_540, 2)
┌───────────────┬───────┐
│ TransactionID ┆ count │
│ ---           ┆ ---   │
│ i64           ┆ u32   │
╞═══════════════╪═══════╡
│ 2987000       ┆ 1     │
│ 2987001       ┆ 1     │
│ 2987002       ┆ 1     │
│ 2987003       ┆ 1     │
│ 2987004       ┆ 1     │
│ …             ┆ …     │
│ 3577535       ┆ 1     │
│ 3577536       ┆ 1     │
│ 3577537       ┆ 1     │
│ 3577538       ┆ 1     │
│ 3577539       ┆ 1     │
└───────────────┴───────┘
number of training data (identity): 144233
shape: (144_233, 2)
┌───────────────┬───────┐
│ TransactionID ┆ count │
│ ---           ┆ ---   │
│ i64           ┆ u32   │
╞═══════════════╪═══════╡
│ 2987004       ┆ 1     │
│ 2987008       ┆ 1     │
│ 2987010       ┆ 1     │
│ 2987011       ┆ 1     │

In [51]:
print("features data")
print("Train Transaction:")

for name, dtype in train_transaction.schema.items():
    print(f"{name}: {dtype}")

print("\nTrain Identity:")
for name, dtype in train_identity.schema.items():
    print(f"{name}: {dtype}")

features_cols = train_transaction.columns[2:] + train_identity.columns[1:]

features data
Train Transaction:
TransactionID: Int64
isFraud: Int64
TransactionDT: Int64
TransactionAmt: Float64
ProductCD: String
card1: Int64
card2: Float64
card3: Float64
card4: String
card5: Float64
card6: String
addr1: Float64
addr2: Float64
dist1: Float64
dist2: Float64
P_emaildomain: String
R_emaildomain: String
C1: Float64
C2: Float64
C3: Float64
C4: Float64
C5: Float64
C6: Float64
C7: Float64
C8: Float64
C9: Float64
C10: Float64
C11: Float64
C12: Float64
C13: Float64
C14: Float64
D1: Float64
D2: Float64
D3: Float64
D4: Float64
D5: Float64
D6: Float64
D7: Float64
D8: Float64
D9: Float64
D10: Float64
D11: Float64
D12: Float64
D13: Float64
D14: Float64
D15: Float64
M1: String
M2: String
M3: String
M4: String
M5: String
M6: String
M7: String
M8: String
M9: String
V1: Float64
V2: Float64
V3: Float64
V4: Float64
V5: Float64
V6: Float64
V7: Float64
V8: Float64
V9: Float64
V10: Float64
V11: Float64
V12: Float64
V13: Float64
V14: Float64
V15: Float64
V16: Float64
V17: Float64
V18: Flo

In [137]:
# identify stringed columns in train_transaction and train_identity
string_cols_transaction = [col for col, x in train_transaction.schema.items() if x == pl.String]
string_cols_identity = [col for col, x in train_identity.schema.items() if x == pl.String]

# Merge all string columns
string_cols = string_cols_transaction + string_cols_identity


label_encoders = {}

for col in string_cols:
    le = LabelEncoder()
    if col in train_transaction.columns:
        train_transaction = train_transaction.with_columns(pl.Series(le.fit_transform(train_transaction[col].fill_null("UNKNOWN"))).alias(col))

    else:
        train_identity = train_identity.with_columns(pl.Series(le.fit_transform(train_identity[col].fill_null("UNKNOWN"))).alias(col))
    
    label_encoders[col] = le

print("Encoding done for string columns below:")   
print("String Columns from transaction:", string_cols_transaction)
print("String Columns from identity:", string_cols_identity)


Encoding done for string columns below:
String Columns from transaction: ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']
String Columns from identity: ['id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']


In [138]:
joblib.dump(label_encoders, "fraud_label_encorders.pkl")

['label_encoders_fraud.pkl']

In [140]:
print(label_encoders)

{'ProductCD': LabelEncoder(), 'card4': LabelEncoder(), 'card6': LabelEncoder(), 'P_emaildomain': LabelEncoder(), 'R_emaildomain': LabelEncoder(), 'M1': LabelEncoder(), 'M2': LabelEncoder(), 'M3': LabelEncoder(), 'M4': LabelEncoder(), 'M5': LabelEncoder(), 'M6': LabelEncoder(), 'M7': LabelEncoder(), 'M8': LabelEncoder(), 'M9': LabelEncoder(), 'id_12': LabelEncoder(), 'id_15': LabelEncoder(), 'id_16': LabelEncoder(), 'id_23': LabelEncoder(), 'id_27': LabelEncoder(), 'id_28': LabelEncoder(), 'id_29': LabelEncoder(), 'id_30': LabelEncoder(), 'id_31': LabelEncoder(), 'id_33': LabelEncoder(), 'id_34': LabelEncoder(), 'id_35': LabelEncoder(), 'id_36': LabelEncoder(), 'id_37': LabelEncoder(), 'id_38': LabelEncoder(), 'DeviceType': LabelEncoder(), 'DeviceInfo': LabelEncoder()}


In [119]:
train_transaction.head()

TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,…,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
i64,i64,i64,f64,i64,i64,f64,f64,i64,f64,i64,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2987000,0,86400,68.5,4,13926,null,150.0,2,142.0,2,315.0,87.0,19.0,null,0,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,null,13.0,null,null,null,…,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987001,0,86401,29.0,4,2755,404.0,150.0,3,102.0,2,325.0,87.0,null,null,17,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,null,null,0.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987002,0,86469,59.0,4,4663,490.0,150.0,4,166.0,3,330.0,87.0,287.0,null,36,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,null,null,0.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987003,0,86499,50.0,4,18132,567.0,150.0,3,117.0,3,476.0,87.0,null,null,54,0,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,null,…,0.0,0.0,1.0,50.0,1758.0,925.0,0.0,354.0,0.0,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987004,0,86506,50.0,1,4497,514.0,150.0,3,102.0,2,420.0,87.0,null,null,17,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,null,null,null,null,null,…,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [155]:
train_identity.head()

TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,i64,i64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64
2987004,0.0,70787.0,null,null,null,null,null,null,null,null,100.0,1,null,-480.0,1,1,166.0,null,542.0,144.0,null,null,3,null,null,null,2,1,1,7,124,32.0,164,4,1,0,1,1,2,954
2987008,-5.0,98945.0,null,null,0.0,-5.0,null,null,null,null,100.0,1,49.0,-300.0,1,1,166.0,null,621.0,500.0,null,null,3,null,null,null,2,1,1,64,99,32.0,48,3,1,0,0,1,2,1728
2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,null,null,0.0,0.0,100.0,1,52.0,null,0,0,121.0,null,410.0,142.0,null,null,3,null,null,null,2,0,0,41,45,null,260,0,0,0,1,1,1,1599
2987011,-5.0,221832.0,null,null,0.0,-6.0,null,null,null,null,100.0,1,52.0,null,1,1,225.0,null,176.0,507.0,null,null,3,null,null,null,2,1,1,41,45,null,260,0,0,0,1,1,1,1557
2987016,0.0,7460.0,0.0,0.0,1.0,0.0,null,null,0.0,0.0,100.0,1,null,-300.0,0,0,166.0,15.0,529.0,575.0,null,null,3,null,null,null,2,0,0,24,45,24.0,40,4,1,0,1,1,1,723


In [141]:
# Memisahkan fitur dan label
X = train_transaction.drop(['isFraud', 'TransactionID'])
y = train_transaction['isFraud']

# Menyeimbangkan data dengan undersampling kelas mayoritas (isFraud=0)
fraud_data = train_transaction.filter(pl.col('isFraud') == 1)
non_fraud_data = train_transaction.filter(pl.col('isFraud') == 0)
print("Before data balancing")
print("amount of fraud data:", fraud_data.shape[0])
print("amount of non-fraud data:", non_fraud_data.shape[0])
# Mengambil sampel dari data non-fraud sebanyak jumlah data fraud
n_fraud = fraud_data.shape[0]
non_fraud_balanced = non_fraud_data.sample(n=n_fraud, seed=42)

# Menggabungkan data fraud dan non-fraud yang sudah diseimbangkan
balanced_data = pl.concat([fraud_data, non_fraud_balanced]).sample(fraction=1.0, seed=42)

# Memisahkan fitur dan label dari data yang sudah diseimbangkan
X = balanced_data.drop(['isFraud', 'TransactionID'])
y = balanced_data['isFraud']
fraud_data = balanced_data.filter(pl.col('isFraud') == 1)
non_fraud_data = balanced_data.filter(pl.col('isFraud') == 0)
print()
print("After data balancing")
print("amount of fraud data:", fraud_data.shape[0])
print("amount of non-fraud data:", non_fraud_data.shape[0])

Before data balancing
amount of fraud data: 20663
amount of non-fraud data: 569877

After data balancing
amount of fraud data: 20663
amount of non-fraud data: 20663


In [142]:


# Membagi data menjadi training dan testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Membuat dan melatih model Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Get predictions
y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)

# Calculate scores
train_score = accuracy_score(y_train, y_train_pred)
test_score = accuracy_score(y_test, y_test_pred)

# Calculate sensitivity (recall) and specificity
train_sensitivity = recall_score(y_train, y_train_pred)
test_sensitivity = recall_score(y_test, y_test_pred)

train_specificity = recall_score(y_train, y_train_pred, pos_label=0)
test_specificity = recall_score(y_test, y_test_pred, pos_label=0)


print(f"accuracy training: {train_score:.4f}")
print(f"accuracy testing: {test_score:.4f}")
print(f"sensitifity training: {train_sensitivity:.4f}")
print(f"sensitifity testing: {test_sensitivity:.4f}")
print(f"spesificity training: {train_specificity:.4f}")
print(f"spesificity testing: {test_specificity:.4f}")


accuracy training: 0.9998
accuracy testing: 0.8487
sensitifity training: 0.9996
sensitifity testing: 0.8311
spesificity training: 0.9999
spesificity testing: 0.8662


In [143]:
import joblib

joblib.dump(rf_model, "fraud_model.pkl")

['fraud_model.pkl']

## Model Evaluation in test data

In [144]:
rf_model = joblib.load("fraud_model.pkl")
label_encoders = joblib.load("fraud_label_encoders.pkl")

In [145]:
label_encoders

{'ProductCD': LabelEncoder(),
 'card4': LabelEncoder(),
 'card6': LabelEncoder(),
 'P_emaildomain': LabelEncoder(),
 'R_emaildomain': LabelEncoder(),
 'M1': LabelEncoder(),
 'M2': LabelEncoder(),
 'M3': LabelEncoder(),
 'M4': LabelEncoder(),
 'M5': LabelEncoder(),
 'M6': LabelEncoder(),
 'M7': LabelEncoder(),
 'M8': LabelEncoder(),
 'M9': LabelEncoder(),
 'id_12': LabelEncoder(),
 'id_15': LabelEncoder(),
 'id_16': LabelEncoder(),
 'id_23': LabelEncoder(),
 'id_27': LabelEncoder(),
 'id_28': LabelEncoder(),
 'id_29': LabelEncoder(),
 'id_30': LabelEncoder(),
 'id_31': LabelEncoder(),
 'id_33': LabelEncoder(),
 'id_34': LabelEncoder(),
 'id_35': LabelEncoder(),
 'id_36': LabelEncoder(),
 'id_37': LabelEncoder(),
 'id_38': LabelEncoder(),
 'DeviceType': LabelEncoder(),
 'DeviceInfo': LabelEncoder()}

In [146]:
for col in string_cols:
    le = label_encoders[col]
    if col in test_transaction.columns:
        # Handle unseen categories by mapping them to "UNKNOWN"
        test_col = test_transaction[col].fill_null("UNKNOWN")
        # Get unique values in test data not present in training
        unseen = set(test_col.unique()) - set(le.classes_)
        # Replace unseen values with "UNKNOWN" 
        test_col = test_col.map_elements(lambda x: "UNKNOWN" if x in unseen else x, return_dtype=pl.Utf8)
        test_transaction = test_transaction.with_columns(pl.Series(le.transform(test_col)).alias(col))

    else:
        # Handle unseen categories by mapping them to "UNKNOWN"
        test_col = test_identity[col].fill_null("UNKNOWN")
        # Get unique values in test data not present in training
        unseen = set(test_col.unique()) - set(le.classes_)
        # Replace unseen values with "UNKNOWN" 
        test_col = test_col.map_elements(lambda x: "UNKNOWN" if x in unseen else x, return_dtype=pl.Utf8)
        test_identity = test_identity.with_columns(pl.Series(le.transform(test_col)).alias(col))


In [153]:
test_transaction.write_csv("ieee-fraud-detection/test_transaction_decoded.csv")

In [148]:
X_valid = test_transaction.drop(['TransactionID'])

y_valid = rf_model.predict(X_valid)

In [151]:
submission = pl.DataFrame({"TransactionID":test_transaction["TransactionID"], "isFraud": y_valid})

In [152]:
submission.write_csv("submission.csv")